In [2]:
import torch
import os
from lib.vit_ct import CRC_model

model = CRC_model("CRCFound_large_patch16_256")
pretrained_path = 'checkpoints/checkpoint-999.pth'  # TODO
if os.path.isfile(pretrained_path):
    print(f"Loading pretrained weights from: {pretrained_path}")
    checkpoint = torch.load(pretrained_path, map_location='cpu')
    pretrained_weights = checkpoint.get('model', checkpoint) 
    cleaned_weights = {k.replace("module.", ""): v for k, v in pretrained_weights.items()}
    compatible_weights = {k: v for k, v in cleaned_weights.items() if k in model.state_dict()}
    model.load_state_dict(
        {**model.state_dict(), **compatible_weights}
    )
    print("Pretrained weights loaded successfully.")
else:
    raise FileNotFoundError(f"Pretrained model not found at: {pretrained_path}")
model.to(device="cuda")
model.eval()

Loading pretrained weights from: checkpoints/checkpoint-999.pth
Pretrained weights loaded successfully.


ViT(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(1, 1024, kernel_size=(16, 16, 16), stride=(16, 16, 16))
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (adapter): Adapter_Layer(
        (adapter_layer_norm_before): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)


In [3]:
import torch
import SimpleITK as sitk
import numpy as np
img_root = '' # TODO
img = sitk.ReadImage(img_root)
img = torch.from_numpy(sitk.GetArrayFromImage(img).astype(np.float32))
img_tensor = img.unsqueeze(0).unsqueeze(0)
with torch.inference_mode():
    image_embeddings = model(
        input=img_tensor.to("cuda"),
        return_global=True    # return global feature token
    )[0]  # shape: [1, feature_dim]